In [17]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical

In [2]:
# lenders
USB= 504713  # US Bank
Wells=451965  #Wells Fargo
Quicken=7197000003        #Quicken Loan

loans_raw = pd.read_csv('AllData2017.csv')
loans_df = loans_raw.loc[(loans_raw['respondent id'] == Wells) | (loans_raw['respondent id'] == USB) | (loans_raw['respondent id'] == Quicken), :]
loans_df.head()

,Index,respondent id,agency code,loan type,property type,loan purpose,owner occ,loan amount,pre approval,action taken,...,denial reason 1,denial reason 2,denial reason 3,rate of spread,hoepa status,lien status,loan type modified,action modified,income cleaned,income loan ratio
0,1,451965,9,1,1,3,1,43,3,1,...,NaN,NaN,NaN,NaN,2,1,1,1,41,105
1,2,451965,9,2,1,1,1,53,3,1,...,NaN,NaN,NaN,NaN,2,1,0,1,0,0
2,3,451965,9,1,1,3,1,114,3,1,...,NaN,NaN,NaN,NaN,2,1,1,1,55,207
3,4,451965,9,1,1,3,1,142,3,1,...,NaN,NaN,NaN,NaN,2,1,1,1,75,189
4,5,451965,9,1,1,1,1,162,3,1,...,NaN,NaN,NaN,NaN,2,1,1,1,0,0


In [3]:
# Assign X (data) and y (target)
#drop all of the columns with non-numeric data
X=loans_df.drop(["action modified","respondent id", "state", "county", "loan type", "owner occ", "census tract", "hoepa status", "Index", "agency code", "income", "msa/md", "pre approval", "action taken", "denial reason 1", "denial reason 2", "rate of spread", "denial reason 3", "type of purchaser"],axis=1)

y=loans_df["action modified"]
print(X.shape, y.shape)
X.head()

(34071, 9) (34071,)


,property type,loan purpose,loan amount,applicant race 1,sex,lien status,loan type modified,income cleaned,income loan ratio
0,1,3,43,5,2,1,1,41,105
1,1,1,53,5,2,1,0,0,0
2,1,3,114,6,3,1,1,55,207
3,1,3,142,3,1,1,1,75,189
4,1,1,162,2,2,1,1,0,0


In [4]:
# Use train_test_split to create training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y)
#X_train.dtypes

In [18]:
X_train_continuous = pd.DataFrame(X_train, columns=['income cleaned', 'loan amount', 'income loan ratio'])
X_test_continuous = pd.DataFrame(X_test, columns=['income cleaned', 'loan amount', 'income loan ratio'])

# Create a StandardScater model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_train_continuous)


In [19]:
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train_continuous)
X_test_scaled = X_scaler.transform(X_test_continuous)

In [8]:
# One-hot encoding
property_type_train = to_categorical(X_train["property type"])
loan_purpose_train = to_categorical(X_train["loan purpose"])
race_train = to_categorical(X_train["applicant race 1"])
sex_train = to_categorical(X_train["sex"])
lien_status_train = to_categorical(X_train["lien status"])
loan_type_train = to_categorical(X_train["loan type modified"])

property_type_test = to_categorical(X_test["property type"])
loan_purpose_test = to_categorical(X_test["loan purpose"])
race_test = to_categorical(X_test["applicant race 1"])
sex_test = to_categorical(X_test["sex"])
lien_status_test = to_categorical(X_test["lien status"])
loan_type_test = to_categorical(X_test["loan type modified"])

C:\Users\llreb\Anaconda3\envs\PythonData\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
categorical_train_X = np.hstack([property_type_train, loan_purpose_train, lien_status_train, loan_type_train])
categorical_test_X = np.hstack([property_type_test, loan_purpose_test, lien_status_test, loan_type_test])
print(categorical_train_X.shape)

(25553, 13)


In [20]:
all_train_X = np.hstack([categorical_train_X, X_train_scaled])
all_test_X = np.hstack([categorical_test_X, X_test_scaled])
print(all_train_X[0])

[0.         1.         0.         0.         1.         0.
 0.         0.         1.         0.         0.         0.
 1.         0.01       0.02734884 0.0313186 ]


In [11]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [12]:
# First, create a normal neural network with 
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=16))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [13]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
# Fit the model to the training data
model.fit(
    all_train_X,
    y_train_categorical,
    epochs=500,
    shuffle=True,
    verbose=2
)

Epoch 1/500
 - 1s - loss: 1.1742 - acc: 0.7618
Epoch 2/500
 - 1s - loss: 1.1692 - acc: 0.7620
Epoch 3/500
 - 1s - loss: 0.5520 - acc: 0.7704
Epoch 4/500
 - 1s - loss: 0.5047 - acc: 0.7708
Epoch 5/500
 - 1s - loss: 0.5038 - acc: 0.7715
Epoch 6/500
 - 1s - loss: 0.5039 - acc: 0.7715
Epoch 7/500
 - 1s - loss: 0.5036 - acc: 0.7705
Epoch 8/500
 - 1s - loss: 0.5026 - acc: 0.7713
Epoch 9/500
 - 1s - loss: 0.5025 - acc: 0.7728
Epoch 10/500
 - 1s - loss: 0.5018 - acc: 0.7713
Epoch 11/500
 - 1s - loss: 0.5007 - acc: 0.7724
Epoch 12/500
 - 1s - loss: 0.4995 - acc: 0.7720
Epoch 13/500
 - 1s - loss: 0.4985 - acc: 0.7725
Epoch 14/500
 - 1s - loss: 0.4968 - acc: 0.7727
Epoch 15/500
 - 1s - loss: 0.4952 - acc: 0.7761
Epoch 16/500
 - 1s - loss: 0.4943 - acc: 0.7759
Epoch 17/500
 - 1s - loss: 0.4939 - acc: 0.7768
Epoch 18/500
 - 1s - loss: 0.4934 - acc: 0.7772
Epoch 19/500
 - 1s - loss: 0.4924 - acc: 0.7776
Epoch 20/500
 - 1s - loss: 0.4923 - acc: 0.7776
Epoch 21/500
 - 1s - loss: 0.4925 - acc: 0.7791
E

Epoch 171/500
 - 1s - loss: 0.4798 - acc: 0.7858
Epoch 172/500
 - 1s - loss: 0.4795 - acc: 0.7851
Epoch 173/500
 - 1s - loss: 0.4799 - acc: 0.7853
Epoch 174/500
 - 1s - loss: 0.4796 - acc: 0.7860
Epoch 175/500
 - 1s - loss: 0.4797 - acc: 0.7847
Epoch 176/500
 - 1s - loss: 0.4794 - acc: 0.7852
Epoch 177/500
 - 1s - loss: 0.4799 - acc: 0.7850
Epoch 178/500
 - 1s - loss: 0.4796 - acc: 0.7843
Epoch 179/500
 - 1s - loss: 0.4799 - acc: 0.7861
Epoch 180/500
 - 1s - loss: 0.4797 - acc: 0.7848
Epoch 181/500
 - 1s - loss: 0.4795 - acc: 0.7849
Epoch 182/500
 - 1s - loss: 0.4794 - acc: 0.7850
Epoch 183/500
 - 1s - loss: 0.4794 - acc: 0.7851
Epoch 184/500
 - 1s - loss: 0.4794 - acc: 0.7849
Epoch 185/500
 - 1s - loss: 0.4798 - acc: 0.7844
Epoch 186/500
 - 1s - loss: 0.4795 - acc: 0.7852
Epoch 187/500
 - 1s - loss: 0.4796 - acc: 0.7850
Epoch 188/500
 - 1s - loss: 0.4793 - acc: 0.7852
Epoch 189/500
 - 1s - loss: 0.4789 - acc: 0.7861
Epoch 190/500
 - 1s - loss: 0.4790 - acc: 0.7853
Epoch 191/500
 - 1s 

 - 1s - loss: 0.4772 - acc: 0.7876
Epoch 339/500
 - 1s - loss: 0.4772 - acc: 0.7865
Epoch 340/500
 - 1s - loss: 0.4773 - acc: 0.7862
Epoch 341/500
 - 1s - loss: 0.4773 - acc: 0.7867
Epoch 342/500
 - 1s - loss: 0.4774 - acc: 0.7863
Epoch 343/500
 - 1s - loss: 0.4773 - acc: 0.7867
Epoch 344/500
 - 1s - loss: 0.4771 - acc: 0.7866
Epoch 345/500
 - 1s - loss: 0.4772 - acc: 0.7863
Epoch 346/500
 - 1s - loss: 0.4776 - acc: 0.7857
Epoch 347/500
 - 1s - loss: 0.4771 - acc: 0.7865
Epoch 348/500
 - 1s - loss: 0.4773 - acc: 0.7861
Epoch 349/500
 - 1s - loss: 0.4768 - acc: 0.7861
Epoch 350/500
 - 1s - loss: 0.4769 - acc: 0.7870
Epoch 351/500
 - 1s - loss: 0.4772 - acc: 0.7869
Epoch 352/500
 - 1s - loss: 0.4772 - acc: 0.7869
Epoch 353/500
 - 1s - loss: 0.4771 - acc: 0.7863
Epoch 354/500
 - 1s - loss: 0.4771 - acc: 0.7869
Epoch 355/500
 - 1s - loss: 0.4771 - acc: 0.7869
Epoch 356/500
 - 1s - loss: 0.4765 - acc: 0.7873
Epoch 357/500
 - 1s - loss: 0.4770 - acc: 0.7880
Epoch 358/500
 - 1s - loss: 0.4770

In [22]:
model_loss, model_accuracy = model.evaluate(
    all_test_X, y_test_categorical, verbose=2)
print(
    f"Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Neural Network - Loss: 0.48275681106969764, Accuracy: 0.7897393754542392


In [ ]:
# pull the test applicants
# 2 - Asian
# 3 - African American
# 5 - white
# sex = 1 male
#sex = 2 female
test_df=loans_df.loc[loans_df["applicant race 1"] == 3,:]
#test_df=loans_df.loc[loans_df["sex"] == 1,:]
test_df.head()

In [ ]:

X_predict=test_df.drop(["action modified","respondent id", "state", "county","census tract", "loan type", "hoepa status", "Index", "agency code", "income", "msa/md", "pre approval", "action taken", "denial reason 1", "denial reason 2", "rate of spread", "denial reason 3", "type of purchaser"],axis=1)
y_predict=test_df["action modified"]
print(X_predict.shape, y_predict.shape)

In [ ]:
# scale the predictive data
#X_predict_scaled = X_scaler.transform(X_predict)
#y_predict_categorical = to_categorical(y_predict)


In [ ]:
#make a prediction and compare it to the actual labels
#predictions = model.predict_classes(X_predict_scaled)
#predict_df = pd.DataFrame({"Prediction": predictions, "Actual": y_predict})
#predict_df.head()


In [ ]:
#model_loss, model_accuracy = model.evaluate(
#    X_predict_scaled, y_predict_categorical, verbose=2)
#print(
#    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
#predict_df.to_csv("predictions2.csv")

In [ ]:
# Calculate confusion matrix
#from sklearn.metrics import confusion_matrix
#confusion_matrix(y_predict, predictions)